In [3]:
##this is to test if results of model are good ennough


from nltk.stem.porter import *
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

import pandas as pd

#import gensim
from gensim import corpora, models, similarities
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

dirc = r'C:\\Users\\DD\\OneDrive\\Documents\\Rapid_Assessment_Tools\\Plans_test\\'

In [4]:
#load file to list 'paragraphs' split by paragraph

f = open(dirc + 'Bhutan_1_utf8.txt', 'r').read().split("\n\n")
f = [i.replace('\n', '') for i in f]

paragraphs = []
for lines in f:
    if len(lines) > 120:
        paragraphs.append(lines)
len(paragraphs)

1331

In [5]:
#tokenization
p_tokenized = [[word.lower() for word in word_tokenize(document.decode('utf-8'))] for document in paragraphs]
len(p_tokenized)

1331

In [6]:
#remove stopwords
english_stopwords = stopwords.words('english')
p_no_stopwords = [[word for word in document if not word in english_stopwords] for document in p_tokenized]
print p_no_stopwords[1220]

[u'targeted', u'poverty', u'reduction', u'interventions', u',', u'four', u'villages', u'namely', u'dangochang', u',', u'chebesa', u',', u'barchong', u'tshochekha', u'supported', u'reap', u'phase', u'ii', u'.', u'income', u'generating', u'sustainable', u'livelihood', u'activities', u',', u'capacity', u'development', u'establishment', u'self-help', u'groups/cooperatives', u'interventions', u'village', u'level', u'additional', u'support', u'household', u'levels', u'provided', u'poorest', u'households', u'within', u'village', u'.', u'funds', u'programme', u'directly', u'provided', u'centre', u'.']


In [7]:
#remove punctuations
english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '...'] #add '...'
p_filtered = [[word for word in document if not word in english_punctuations] for document in p_no_stopwords]
print p_filtered[1220]

[u'targeted', u'poverty', u'reduction', u'interventions', u'four', u'villages', u'namely', u'dangochang', u'chebesa', u'barchong', u'tshochekha', u'supported', u'reap', u'phase', u'ii', u'income', u'generating', u'sustainable', u'livelihood', u'activities', u'capacity', u'development', u'establishment', u'self-help', u'groups/cooperatives', u'interventions', u'village', u'level', u'additional', u'support', u'household', u'levels', u'provided', u'poorest', u'households', u'within', u'village', u'funds', u'programme', u'directly', u'provided', u'centre']


In [9]:
#stemming
p_stemmed = [[st.stem(word) for word in docment] for docment in p_filtered]
print p_stemmed[1220]

[u'target', u'poverty', u'reduc', u'interv', u'four', u'vil', u'nam', u'dangochang', u'chebes', u'barchong', u'tshochekha', u'support', u'reap', u'phas', u'ii', u'incom', u'gen', u'sustain', u'liv', u'act', u'capac', u'develop', u'est', u'self-help', u'groups/cooperative', u'interv', u'vil', u'level', u'addit', u'support', u'household', u'level', u'provid', u'poorest', u'household', u'within', u'vil', u'fund', u'program', u'direct', u'provid', u'cent']


In [13]:
#remove word with low frequency
all_stems = sum(p_stemmed, [])
stems_once = set(stem for stem in set(all_stems) if all_stems.count(stem) == 1)
texts = [[stem for stem in text if stem not in stems_once] for text in p_stemmed]

In [14]:
######## load goals

#use SDG as query

targets = pd.read_excel(dirc +'SDG sub goals.xlsx', names = ['ID', 'Target'],sheetname = 'Sheet1')
targets.head()

,ID,Target
0,1.1,"1.1 By 2030, eradicate extreme poverty for all..."
1,1.2,"1.2 By 2030, reduce at least by half the propo..."
2,1.3,1.3 Implement nationally appropriate social pr...
3,1.4,"1.4 By 2030, ensure that all men and women, in..."
4,1.5,"1.5 By 2030, build the resilience of the poor ..."


In [15]:
#make a list with goal contents

goal = []
for doc in targets.iloc[:]['Target']:
    goal.append(doc)

goal_id = targets.iloc[:]['ID']
goal[10] #test goal 2.4
#goal_id[10]

u'2.4 By 2030, ensure sustainable food production systems and implement resilient agricultural practices that increase productivity and production, that help maintain ecosystems, that strengthen capacity for adaptation to climate change, extreme weather, drought, flooding and other disasters and that progressively improve land and soil quality\xa0'

In [16]:
#tokenization
goal_tokenized = [[word.lower() for word in word_tokenize(document)] for document in goal]

#remove stopwords
goal_no_sw = [[word for word in document if not word in english_stopwords] for document in goal_tokenized]

#stemming
goal_stemmed = [[st.stem(word) for word in l] for l in goal_no_sw]
len(goal_stemmed[10])

38

In [17]:
g_query = goal_stemmed #_frequent
len(g_query[10])

38

In [ ]:
####### match

In [30]:
def match_goal(texts, query, num=100):
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=num)
    index = similarities.MatrixSimilarity(lsi[corpus])
    ml_bhutan = query
    ml_bow = dictionary.doc2bow(ml_bhutan)
    ml_lsi = lsi[ml_bow]
    sims = index[ml_lsi]
    sort_sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print sort_sims[0:100]

In [31]:
match_goal(texts, g_query[10])

2017-02-10 00:26:05,509 : INFO : adding document #0 to Dictionary(0 unique tokens: [])


2017-02-10 00:26:05,651 : INFO : built Dictionary(3046 unique tokens: [u'lhuents', u'1200', u'bear', u'four', u'phajod']...) from 1331 documents (total 56199 corpus positions)


2017-02-10 00:26:05,756 : INFO : collecting document frequencies


2017-02-10 00:26:05,759 : INFO : PROGRESS: processing document #0


2017-02-10 00:26:05,796 : INFO : calculating IDF weights for 1331 documents and 3045 features (44303 matrix non-zeros)


...

2017-02-10 00:26:06,328 : INFO : topic #3(4.074): 0.243*"perc" + -0.193*"farm" + 0.192*"vil" + -0.182*"market" + -0.179*"facil" + -0.172*"produc" + -0.162*"agricult" + -0.151*"livestock" + 0.150*"level" + -0.142*"tour"


2017-02-10 00:26:06,332 : INFO : topic #4(3.873): -0.291*"proport" + -0.243*"pop" + 0.175*"vil" + -0.172*"ii" + -0.168*"development°±" + -0.167*"°∞self-reliance" + -0.160*"tour" + 0.154*"support" + -0.154*"keep" + 0.139*"market"


2017-02-10 00:26:06,339 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


2017-02-10 00:26:06,486 : INFO : creating matrix with 1331 documents and 100 features


[(1200, 0.64878607), (1073, 0.64179593), (955, 0.59496409), (888, 0.58894688), (1247, 0.58887935), (913, 0.58774322), (961, 0.57525998), (546, 0.57305968), (1100, 0.5719372), (1205, 0.56955922), (1258, 0.54628813), (1223, 0.52869928), (1263, 0.5251528), (893, 0.51373851), (851, 0.51371974), (531, 0.50209439), (985, 0.49586684), (592, 0.49570248), (873, 0.49462157), (146, 0.492322), (547, 0.48060277), (981, 0.4805066), (963, 0.4751941), (1044, 0.47515059), (548, 0.4746733), (536, 0.47387207), (794, 0.47316504), (1265, 0.46534264), (1242, 0.4641923), (1081, 0.45105031), (1029, 0.44932601), (568, 0.44407195), (847, 0.43337983), (865, 0.43256277), (939, 0.42699033), (1215, 0.42292976), (1142, 0.42165905), (957, 0.41832817), (796, 0.41700244), (544, 0.41681421), (1054, 0.4165116), (277, 0.41592604), (930, 0.41333434), (321, 0.41274351), (448, 0.40837723), (209, 0.40825102), (1079, 0.40667164), (609, 0.40137777), (60, 0.40118441), (1147, 0.39679193), (362, 0.39623559), (1278, 0.3955735), (11

In [24]:
paragraphs[1200] #ranking 1

'Trongsa has suitable climatic conditions which is conducive for enhancing agricultural and horticultural products, besides livestock products such as poultry, piggery and dairy backyard farming. '

In [25]:
paragraphs[147] #ranking 100

'With a strong emphasis on improving the health and education outcomes, achieving progress in the HAI may not be a major constraint other than improving the adult literacy rate which may pose a challenge. However, improving the EVI will require special efforts particularly in addressing economic imbalances, productive capacities and diversification challenges. This will not be easy. Hence, the goodwill, support and cooperation of our development partner in the coming years will be critical for Bhutan. Furthermore, even beyond 2020, Bhutan may require continued cooperation and support from its development partners to ensure that the transition will help establish a solid economic foundation that is dynamic and resilient. '

In [26]:
#testing goal 2.4
goal[10]
#goal_id[10]

u'2.4 By 2030, ensure sustainable food production systems and implement resilient agricultural practices that increase productivity and production, that help maintain ecosystems, that strengthen capacity for adaptation to climate change, extreme weather, drought, flooding and other disasters and that progressively improve land and soil quality\xa0'